Revised file was created on Mon Dec 30 14:00:00 2024

Author: Nakyung Lee, University of Houston; Seán Kavanagh, University of Birmingham

This code includes three different models.
1. revised relative permittivity prediction
2. revised centroide shift prediction
3. 5d1 Ce prediction 

### Descriptor for relative permittivity, centroid shift, and 5d1 Ce models

In [15]:
import warnings
import numpy as np
import pandas as pd
import os
from scipy.spatial import ConvexHull, QhullError
from pymatgen.core.composition import Composition
from pymatgen.core.periodic_table import Species, Element
from pymatgen.core.periodic_table import Specie
from pymatgen.core.structure import Molecule
from pymatgen.analysis.local_env import CrystalNN
from pymatgen.analysis.ewald import EwaldSummation
from pymatgen.analysis.structure_analyzer import SpacegroupAnalyzer
from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import LocalGeometryFinder
from pymatgen.symmetry.analyzer import PointGroupAnalyzer
from tqdm import tqdm
from mp_api.client import MPRester
from pymatgen.io.cif import CifParser
from pymatgen.transformations.advanced_transformations import OrderDisorderedStructureTransformation


### Cation Site Features

In [16]:
polar_pointgroups = ["1","2","m","mm2","3","3m","4","4m","6","6mm"] 
average_anion_polarizability = {"F": 0.634, "Cl": 2.2, "Br": 3.1, "I": 5, "O": 0.793, "S": 2.9, "Se": 3.8, "N": 1.1}
cations_of_interest = ["Na+","K+","Rb+","Cs+","Ca2+","Sr2+","Ba2+","Y3+","Ce4+","La3+","Pr3+",
                       "Nd3+","Sm3+","Eu3+","Gd3+","Tb3+","Dy3+","Er3+","Tm3+","Yb3+","Lu3+"] 
anions_of_interest = ["O2-", "F-", "Cl-", "Br-", "I-", "S2-", "N3-"]

In [35]:
def _custom_formatwarning(msg, *args, **kwargs):
    return f"{msg}\n"
warnings.formatwarning = _custom_formatwarning

def polyhedron_volume(vertices):
    if len(set(tuple(point) for point in vertices)) < 3:
        return 0
    try:
        hull = ConvexHull(vertices)
        return hull.volume
    except Exception as e:
        return 0

# Function to convert integer to Roman values, for extracting Shannon radii
def roman(number):
    num = [1, 4, 5, 9, 10, 40, 50, 90, 100, 400, 500, 900, 1000]
    sym = ["I", "IV", "V", "IX", "X", "XL", "L", "XC", "C", "CD", "D", "CM", "M"]
    i = 12
    roman_string = ""

    while number:
        div = number // num[i]
        number %= num[i]

        while div:
            roman_string += sym[i]
            div -= 1
        i -= 1

    return roman_string

In [36]:
cnn = CrystalNN(cation_anion=True)

def get_polyhedron_dict(structure, site_index, cnn=cnn, dopant_species="Ce3+", verbose=False):
    structure_w_oxi = structure.copy()
    structure_w_oxi.add_oxidation_state_by_guess(max_sites=-1)  # add oxidation states to structure

    polyhedron_dict = {}

    nn_info_dict = cnn.get_nn_info(structure_w_oxi, site_index)
    polyhedron_dict["coordination_number"] = cnn.get_cn(structure_w_oxi, site_index)

    # get ionic radii of central atom and dopant:
    cation_site = structure_w_oxi.sites[site_index]
    cation_specie = cation_site.specie
    try:
        cation_ionic_radius = cation_specie.get_shannon_radius(
            cn=roman(polyhedron_dict["coordination_number"]), radius_type="ionic"
        )
    except KeyError:
        warnings.warn(
            f"No tabulated Shannon radius for {cation_specie} with coordination number {polyhedron_dict['coordination_number']}. Using CN-independent (average) ionic radius instead."
        )
        cation_ionic_radius = cation_site.specie.ionic_radii.get(
            cation_specie.oxi_state, None
        )
    if cation_ionic_radius is None:
        warnings.warn(
            f"No tabulated ionic radius for cation {cation_site.specie} with oxidation state {guessed_oxi_state:+d}. Using average ionic radius instead."
        )
        cation_ionic_radius = cation_site.specie.average_ionic_radius
    dopant_site = structure_w_oxi.sites[site_index]
    dopant_specie = Specie.from_str(dopant_species)
    try:
        dopant_ionic_radius = dopant_specie.get_shannon_radius(
            cn=roman(polyhedron_dict["coordination_number"]), radius_type="ionic"
        )
    except KeyError:
        warnings.warn(
            f"No tabulated Shannon radius for {dopant_specie} with coordination number {polyhedron_dict['coordination_number']}. Using CN-independent (average) ionic radius instead."
        )
        dopant_ionic_radius = dopant_site.specie.ionic_radii.get(
            dopant_specie.oxi_state, None
        )
    if dopant_ionic_radius is None:
        warnings.warn(
            f"No tabulated ionic radius for dopant {dopant_specie} with oxidation state {dopant_specie.oxi_state:+f}. Using average ionic radius instead."
        )
        dopant_ionic_radius = dopant_site.specie.average_ionic_radius

    polyhedron_dict["cation_ionic_radius"] = cation_ionic_radius  # Angstrom
    polyhedron_dict["dopant_ionic_radius"] = dopant_ionic_radius  # Angstrom
    polyhedron_dict["delta_radius"] = (
        cation_ionic_radius - dopant_ionic_radius
    )  # Angstrom

    # get min, max and mean of metal-ligand bond lengths:
    bond_lengths = []  # Bond Lengths
    for i in nn_info_dict:
        bond_lengths.append(
            {
                "Element": i["site"].specie.as_dict()["element"],
                "Distance": f"{i['site'].distance(cation_site):.3f}",
            }
        )
    bond_lengths.sort(key=lambda x: x["Distance"])
    polyhedron_dict["Avg_bond_length"] = np.mean(
        [float(bond_length["Distance"]) for bond_length in bond_lengths]
    )  # Angstrom

    return polyhedron_dict

lgf = LocalGeometryFinder()

def get_chemenv_info(structure, isite):
    lgf.setup_structure(structure=structure)
    coord_envs = lgf.compute_coordination_environments(structure)
    if len(coord_envs[isite]) > 1:
        # choose the one with the higher ce_fraction:
        coord_envs[isite] = sorted(
            coord_envs[isite], key=lambda x: x["ce_fraction"], reverse=True
        )

    try:
        chemenv_dict = {
            #"ce_symbol": coord_envs[isite][0]["ce_symbol"],
                        "chemenv_CN": coord_envs[isite][0]["ce_symbol"].split(":")[1]}
    except IndexError:
        chemenv_dict = {"chemenv_CN": None}

    return chemenv_dict


### Crystal Structure Features

In [37]:
def get_struc_dict(structure):
    struc_dict = {}
    sga = SpacegroupAnalyzer(structure)
    conv_struc = sga.get_conventional_standard_structure()

    struc_dict["SGR No."] = sga.get_space_group_number()
    struc_dict["volume"] = conv_struc.volume  # Angstrom^3
    struc_dict["density"] = conv_struc.density
    struc_dict["volume_per_Z"] = conv_struc.volume / conv_struc.composition\
        .get_reduced_composition_and_factor()[1]  # Angstrom^3
    struc_dict["a"] = conv_struc.lattice.a  # Angstrom
    struc_dict["b"] = conv_struc.lattice.b  # Angstrom
    struc_dict["c"] = conv_struc.lattice.c
    struc_dict["alpha"] = conv_struc.lattice.alpha
    struc_dict["beta"] = conv_struc.lattice.beta
    struc_dict["gamma"] = conv_struc.lattice.gamma  # degrees
    struc_dict["inversion center"] = sga.is_laue()
    struc_dict["inversion center"] = int(struc_dict["inversion center"])
    struc_dict["polar axis"] = sga.get_point_group_symbol() in polar_pointgroups
    struc_dict["polar axis"] = int(struc_dict["polar axis"])

    struc_dict["a/b"] = conv_struc.lattice.a / conv_struc.lattice.b
    struc_dict["b/c"] = conv_struc.lattice.b / conv_struc.lattice.c
    struc_dict["c/a"] = conv_struc.lattice.c / conv_struc.lattice.a
    struc_dict["alpha/beta"] = conv_struc.lattice.alpha / conv_struc.lattice.beta
    struc_dict["beta/gamma"] = conv_struc.lattice.beta / conv_struc.lattice.gamma
    struc_dict["gamma/alpha"] = conv_struc.lattice.gamma / conv_struc.lattice.alpha
    struc_dict["volume_rp"] = conv_struc.volume /10**3
    struc_dict["volume_per_Z_rp"] = conv_struc.volume / conv_struc.composition\
        .get_reduced_composition_and_factor()[1] / 10**3
    struc_dict["volume_per_atom"] = (conv_struc.volume / conv_struc.composition.num_atoms) / 10**3  # Angstrom^3

    structure.add_oxidation_state_by_guess(max_sites=-1)
    struc_dict["Avg. cation electronegativity"] = np.mean([
        Element(site.specie.element).X
        for site in structure.sites
        if site.specie.oxi_state >= 0
    ])
    struc_dict["Avg. anion polarizability"] = np.mean([
        average_anion_polarizability[site.specie.element.symbol]
        for site in structure.sites
        if site.specie.oxi_state < 0
    ])

    num_anions = len([site for site in structure.sites if site.specie.oxi_state < 0])
    num_cations = len([site for site in structure.sites if site.specie.oxi_state >= 0])
    struc_dict["Condensation"] = num_anions / num_cations

    return struc_dict


### Load Data from Materials Project

In [38]:
initial_df = pd.read_excel(r'Tester.xlsx')
initial_formulae = list(initial_df['Composition'])
initial_formulae = [
    Composition(formula).get_integer_formula_and_factor()[0]
    for formula in initial_formulae
]

materials_fields = [
    "material_id",
    "formula_pretty",
    "structure",
    "symmetry",
    "volume",
    "density"
]

with MPRester("SA8tRgHQuURU1rXg822UJpyaCHNVRuKv") as mpr:
    docs = mpr.materials.search(formula=initial_formulae, fields=materials_fields)

Feature_dicts = []

for doc in tqdm(docs):
    try:
        doc.structure.add_oxidation_state_by_guess(max_sites=-1)
        sga = SpacegroupAnalyzer(doc.structure)
        inequivalent_site_indices = set(sga.get_symmetry_dataset()["equivalent_atoms"])
        inequivalent_cation_site_indices = [
            i
            for i in inequivalent_site_indices
            if str(doc.structure.sites[i].specie) in cations_of_interest
        ]
    
        for cation_site_index in inequivalent_cation_site_indices:
            cation_specie = doc.structure.sites[cation_site_index].specie

            Feature_dicts += [
                {
                    "Composition": str(doc.formula_pretty),
                    "Database IDs": doc.material_id,
                    "Central Cation": str(doc.structure.sites[cation_site_index].specie),
                    **get_polyhedron_dict(doc.structure, cation_site_index),
                    **get_chemenv_info(doc.structure, cation_site_index),
                    **get_struc_dict(doc.structure)
                }
            ]    

    except IndexError:
        print(f"skipping composiiton {doc.formula_pretty} due to IndexError")
        continue

temp_df = pd.DataFrame(Feature_dicts)
temp_df["chemenv_CN"] = temp_df["chemenv_CN"].fillna(temp_df["coordination_number"])

Retrieving MaterialsDoc documents:   0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:09,  3.17s/it]No tabulated Shannon radius for La3+ with coordination number 3. Using CN-independent (average) ionic radius instead.
No tabulated Shannon radius for Ce3+ with coordination number 3. Using CN-independent (average) ionic radius instead.
100%|██████████| 4/4 [00:08<00:00,  2.03s/it]


In [40]:
rp_columns = [
    'Composition',
    'Database IDs',
    'Central Cation',
    'SGR No.',
    'volume_rp',
    'density',
    'a/b',
    'b/c',
    'c/a',
    'alpha/beta',
    'beta/gamma',
    'gamma/alpha',
    'inversion center',
    'polar axis',
    'volume_per_Z_rp',
    'volume_per_atom'
]

cs_columns = [
    'Composition',
    'Database IDs',
    'Central Cation',
    'coordination_number',
    'cation_ionic_radius',
    'delta_radius',
    'Avg_bond_length',
    'Avg. cation electronegativity',
    'Avg. anion polarizability',
    'Condensation'
]

d1_columns = [
    'Composition',
    'Database IDs',
    'Central Cation',
    'coordination_number',
    'cation_ionic_radius',
    'dopant_ionic_radius',
    'chemenv_CN',
    'SGR No.',
    'volume',
    'volume_per_Z',
    'a',
    'b',
    'gamma'
]

RP_df = temp_df[rp_columns]
CS_df = temp_df[cs_columns]
d1_df = temp_df[d1_columns]

### Load Data from cif Files for Disordered Structures

This part is an alternative for disordered structures or structures not in Materials Project

In [ ]:
def _custom_formatwarning(msg, *args, **kwargs):
    return f"{msg}\n"
warnings.formatwarning = _custom_formatwarning

cif_directory  = os.getcwd()
Feature_dicts = []
# put the cif files in the same folder where this code exist
for cif_filename in os.listdir(cif_directory):
    if cif_filename.endswith(".cif"):
        cif_file_path = os.path.join(cif_directory, cif_filename)

        try:
            # Parse the CIF file
            with open(cif_file_path, "r") as cif_file:
                cif_parser = CifParser(cif_file)
                structure = cif_parser.parse_structures(primitive=True)[0] 

            odt = OrderDisorderedStructureTransformation()
            ordered_structures = odt.apply_transformation(structure, return_ranked_list=True)

            if ordered_structures:
                structure = ordered_structures[0]["structure"]
            
            structure.add_oxidation_state_by_guess(max_sites=-1)
            sga = SpacegroupAnalyzer(structure)
            inequivalent_site_indices = set(sga.get_symmetry_dataset()["equivalent_atoms"])
            inequivalent_cation_site_indices = [
                i
                for i in inequivalent_site_indices
                if str(structure.sites[i].specie) in cations_of_interest
            ]
    
            for cation_site_index in inequivalent_cation_site_indices:
                cation_specie = structure.sites[cation_site_index].specie

                Feature_dicts += [
                    {
                        "File name": cif_filename,
                        "Central Cation": str(structure.sites[cation_site_index].specie),
                        **get_polyhedron_dict(structure, cation_site_index),
                        **get_chemenv_info(structure, cation_site_index),
                        **get_struc_dict(structure)
                    }
                ]    

        except Exception as e:
            print(f"Error processing {cif_filename}: {e}")

temp_df = pd.DataFrame(Feature_dicts)
temp_df["chemenv_CN"] = temp_df["chemenv_CN"].fillna(temp_df["coordination_number"])

### Get the Relative Permittivity

In [ ]:

permittivity_df = RP_df[["Composition"]]

class Vectorize_Formula:
    def __init__(self):
        elem_dict = pd.read_excel(r'elements_rp.xlsx') # CHECK NAME OF FILE
        self.element_df = pd.DataFrame(elem_dict)
        self.element_df.set_index('Symbol',inplace=True)
        self.column_names = []
        for string in ['avg','diff','max','min','std']:
            for column_name in list(self.element_df.columns.values):
                self.column_names.append(f'{string}_{column_name}')

    def get_features(self, formula):
        try:
            fractional_composition = Composition(formula).fractional_composition.as_dict()
            avg_feature = np.zeros(len(self.element_df.columns))

            for key in fractional_composition:
                if key not in self.element_df.index:
                    print('The element:', key, 'from formula', formula,'is not currently supported in our database')
                    return np.full(len(self.element_df.colums)*5, np.nan)
                avg_feature += self.element_df.loc[key].values * fractional_composition[key]

            elements_in_formula = list(fractional_composition.keys())
            element_stats = self.element_df.loc[elements_in_formula]
            diff_feature = element_stats.max() - element_stats.min()
            max_feature = element_stats.max()
            min_feature = element_stats.min()
            std_feature = element_stats.std(ddof=0)

            features = np.concatenate([avg_feature, diff_feature, max_feature, min_feature, std_feature])
            return features
        except Exception as e:
            print(f"An error occurred while processing the formula {formula}: {e}")
            return np.full(len(self.element_df.columns) * 5, np.nan)

gf=Vectorize_Formula()

# empty list for storage of bnmmbc
features=[]

# add values to list using for loop
for formula in permittivity_df["Composition"]:
    features.append(gf.get_features(formula))

# feature vectors and targets as X and y
X = pd.DataFrame(features, columns = gf.column_names)
RP_df_first_three = RP_df.iloc[:, :3]
RP_df_rest = RP_df.iloc[:, 3:]
RP_Feature_Set=pd.concat([RP_df_first_three, X, RP_df_rest], axis=1)
RP_Feature_Set.to_excel('RP_feature_set_for_Tester.xlsx', index=False)

### Revised Relative Permittivity Model

In [49]:
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

RPDB = pd.read_excel(r'Training_Set_for_RP.xlsx')
array = RPDB.values
X = array[:,2:100]
Y = array[:,1]
Compd = array[:,0]

In [50]:
RP_model = XGBRegressor(tree_method='hist', device='cuda',  n_estimators=900, learning_rate=0.06,
                          max_depth=5, min_child_weight=9, subsample=0.6, base_score=0.5,
                          colsample_bytree=0.5, colsample_bylevel=1, colsample_bynode=1,
                          reg_alpha=0, reg_lambda=1
                          )
kf = KFold(n_splits=10, shuffle=True, random_state=42)
cv_results = cross_val_score(RP_model, X, Y, cv=kf, scoring='r2')

print("\nCross-validation results for each fold:")
for i, score in enumerate(cv_results, 1):
    print(f"Fold {i}: R2 = {score:.4f}")

print(f"\nMean R2: {np.mean(cv_results):.4f}")
print(f"Standard Deviation of R2: {np.std(cv_results):.4f}")


Cross-validation results for each fold:
Fold 1: R2 = 0.9211
Fold 2: R2 = 0.8558
Fold 3: R2 = 0.9275
Fold 4: R2 = 0.8831
Fold 5: R2 = 0.8646
Fold 6: R2 = 0.8760
Fold 7: R2 = 0.8545
Fold 8: R2 = 0.8811
Fold 9: R2 = 0.8724
Fold 10: R2 = 0.8876

Mean R2: 0.8824
Standard Deviation of R2: 0.0235


In [60]:
RP_model.fit(X,Y)
prediction = pd.read_excel(r'RP_feature_set_for_Tester.xlsx')
a = prediction.values
b = a[:,3:101]
result = RP_model.predict(b)
composition=prediction['Composition']
result=pd.DataFrame(result, columns=['Predicted RP'])
predicted=np.column_stack((composition,result))
predicted_RP=pd.DataFrame(predicted)
predicted_RP.to_excel('Predicted_RP_for_Tester.xlsx', index=False, header=("Composition","Predicted RP"))

CS_df_left = CS_df.iloc[:, :3]
CS_df_right = CS_df.iloc[:, 3:]
CS_feature_set_df = pd.concat([CS_df_left, result, CS_df_right], axis=1)
CS_feature_set_df.to_excel('CS_feature_set_for_Tester.xlsx', index=False)

d1_df_left = d1_df.iloc[:, :3]
d1_df_right = d1_df.iloc[:, 3:]
d1_with_RP_df = pd.concat([d1_df_left, result, d1_df_right], axis=1)

### Predict Centroid Shift

In [62]:
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import numpy as np
import pandas as pd

CSDB = pd.read_excel('Training_Set_for_CS.xlsx')
array = CSDB.values
X = array[:,2:10]
Y = array[:,1]
Compd = array[:,0]

#X_train, X_test, Y_train, Y_test = train_test_split (X,Y, test_size=0.2, random_state=22, shuffle=True)
#X_tr, X_val, Y_tr, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=10)

In [66]:
CS_model = XGBRegressor(tree_method='hist', device='cuda', n_estimators=150, learning_rate=0.05,
                           max_depth=3, min_child_weight=1, subsample=0.6,
                           colsample_bytree=0.9, colsample_bylevel=1, colsample_bynode=0.9,
                           reg_alpha=0.25, reg_lambda=0)
kf = KFold(n_splits=10, shuffle=True, random_state=22)
cv_results = cross_val_score(CS_model, X, Y, cv=kf, scoring='r2')

print("\nCross-validation results for each fold:")
for i, score in enumerate(cv_results, 1):
    print(f"Fold {i}: R2 = {score:.4f}")

print(f"\nMean R2: {np.mean(cv_results):.4f}")
print(f"Standard Deviation of R2: {np.std(cv_results):.4f}")


Cross-validation results for each fold:
Fold 1: R2 = 0.7404
Fold 2: R2 = 0.8558
Fold 3: R2 = 0.9627
Fold 4: R2 = 0.8985
Fold 5: R2 = 0.8846
Fold 6: R2 = 0.8847
Fold 7: R2 = 0.8875
Fold 8: R2 = 0.9672
Fold 9: R2 = 0.9373
Fold 10: R2 = 0.9276

Mean R2: 0.8946
Standard Deviation of R2: 0.0619


In [70]:
CS_model.fit(X,Y)
prediction = pd.read_excel(r'CS_feature_set_for_Tester.xlsx')
a = prediction.values
b = a[:,3:11]
result = CS_model.predict(b)
composition=prediction['Composition']
result=pd.DataFrame(result, columns=['Predicted CS'])
predicted=np.column_stack((composition,result))
predicted_CS=pd.DataFrame(predicted)
predicted_CS.to_excel('Predicted_CS_for_Tester.xlsx', index=False, header=("Composition","Predicted CS"))

d1_with_RP_left = d1_with_RP_df.iloc[:, :3]
d1_with_RP_right = d1_with_RP_df.iloc[:, 3:]
d1_feature_set_df = pd.concat([d1_with_RP_left, result, d1_with_RP_right], axis=1)
d1_feature_set_df.to_excel('d1_wo_compositional_for_Tester.xlsx', index=False)

### Predict 5d1 of Ce3+

In [78]:
composition_df = pd.read_excel(r"d1_wo_compositional_for_Tester.xlsx")

class Vectorize_Formula:
    def __init__(self):
        elem_dict = pd.read_excel(r'elements_5d1.xlsx') # CHECK NAME OF FILE
        self.element_df = pd.DataFrame(elem_dict)
        self.element_df.set_index('Symbol',inplace=True)
        self.column_names = []
        for string in ['avg','diff','max','min','std']:
            for column_name in list(self.element_df.columns.values):
                self.column_names.append(f'{string}_{column_name}')

    def get_features(self, formula):
        try:
            fractional_composition = Composition(formula).fractional_composition.as_dict()
            avg_feature = np.zeros(len(self.element_df.columns))

            for key in fractional_composition:
                if key not in self.element_df.index:
                    print('The element:', key, 'from formula', formula,'is not currently supported in our database')
                    return np.full(len(self.element_df.colums)*5, np.nan)
                avg_feature += self.element_df.loc[key].values * fractional_composition[key]

            elements_in_formula = list(fractional_composition.keys())
            element_stats = self.element_df.loc[elements_in_formula]
            diff_feature = element_stats.max() - element_stats.min()
            max_feature = element_stats.max()
            min_feature = element_stats.min()
            std_feature = element_stats.std(ddof=0)

            features = np.concatenate([avg_feature, diff_feature, max_feature, min_feature, std_feature])
            return features
        except Exception as e:
            print(f"An error occurred while processing the formula {formula}: {e}")
            return np.full(len(self.element_df.columns) * 5, np.nan)

gf=Vectorize_Formula()

# empty list for storage of bnmmbc
features=[]

# add values to list using for loop
for formula in composition_df["Composition"]:
    features.append(gf.get_features(formula))

# feature vectors and targets as X and y
X = pd.DataFrame(features, columns = gf.column_names)
columns_to_extract = [1, 3, 4, 5, 7, 8, 9, 10, 12, 13, 15, 23, 24, 29, 30, 32, 34, 40, 52, 53, 66, 72, 74, 76, 78, 79, 80, 83, 84, 87, 89, 93]
X_extracted = X.iloc[:, columns_to_extract]
completed=pd.concat([composition_df, X_extracted], axis=1)
completed.to_excel('5d1_feature_set_for_Tester.xlsx', index=False)

In [79]:
FDODB=pd.read_excel('Training_Set_for_5d1.xlsx')
array = FDODB.values
X = array[:,2:46]
Y = array[:,1]
Compd = array[:,0]


In [80]:
from sklearn.model_selection import LeaveOneGroupOut

best_model = XGBRegressor(tree_method='hist', device='cuda', n_estimators=200, learning_rate=0.06,
                          max_depth=9, min_child_weight=8, subsample=0.6, base_score=0.4,
                          colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1,
                          reg_alpha=0, reg_lambda=1
                          )

logo=LeaveOneGroupOut()
# Map group labels to integers
unique_groups = np.unique(Compd)
group_to_index = {group: idx for idx, group in enumerate(unique_groups)}

all_Y_val = []
all_Y_pred = []
mse_values = []
rmse_values = []
mae_values = []

for train_index, test_index in logo.split(X, Y, Compd):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

    best_model.fit(X_train, Y_train)
    Y_pred = best_model.predict(X_test)

    mse = mean_squared_error(Y_test, Y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(Y_test, Y_pred)

    current_group_label = Compd[test_index][0]

    mse_values.append(mse)
    rmse_values.append(rmse)
    mae_values.append(mae)

    all_Y_val.extend(Y_test)
    all_Y_pred.extend(Y_pred)

avg_mse = np.mean(mse_values)
avg_rmse = np.mean(rmse_values)
avg_mae = np.mean(mae_values)
print(avg_mae, avg_mse, avg_rmse, r2_score(all_Y_val, all_Y_pred))

0.15338832561753013 0.04344782809517265 0.15425705564280934 0.8380042800908163


In [85]:
best_model.fit(X,Y)
prediction = pd.read_excel(r'5d1_feature_set_for_Tester.xlsx')
a = prediction.values
b = a[:,3:48]
result = best_model.predict(b)
composition=prediction['Composition']
result=pd.DataFrame(result)
divider = 1.23984193 * 10**3
result_in_nm = result.map(lambda x: round(divider/x, 2))
predicted=np.column_stack((composition,result,result_in_nm))
predicted=pd.DataFrame(predicted)
predicted.to_excel('Predicted_5d1_for_Tester.xlsx', index=False, header=("Composition","Predicted 5d1 for Ce3+","Pred 5d1 in nm"))